# *Selenium의 사용을 위해 ChromeDriver를 사용합니다.*
# *ChromeDriver는 Colab에서 사용이 불가능하므로 Jupyter notebook을 사용해야 합니다.*

### 모듈 import

In [2]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

한글 이외의 글자 지우기

In [4]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

## 1. 꿈길 크롤링

In [6]:
# ChromeDriver 경로
driver_path = './chromedriver.exe'

# ChromeDriver 실행
driver = webdriver.Chrome(service=Service(executable_path=driver_path))

# 웹 페이지 접속
url = 'https://www.ggoomgil.go.kr/front/exp/infraMapInfo.do'
driver.get(url)

WebDriverException: ignored

### 필요한 데이터 수작업으로 상세 설정
* 지역 : 경상북도 설정
* 초중고 : 중학교 설정
* 체험일 : 2022년 데이터가 필요하므로 체험 시작일과 종료일 설정, 상관없음 체크 해제

In [ ]:
programs = []

In [ ]:
for p in range(2,50):
    if p > 8:
        nxtpage = driver.find_element(By.XPATH, f'//*[@id="programListPagination"]/div/div/ul/li[7]')
    else:
        nxtpage = driver.find_element(By.XPATH, f'//*[@id="programListPagination"]/div/div/ul/li[{p}]')
    nxtpage.click()
    time.sleep(0.3)
    for i in range(1,7):
        driver.find_element(By.XPATH, f'//*[@id="programList"]/li[{i}]/a').click()
        time.sleep(0.5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        program = {}
        program["title"] = soup.select('body > div.lay-body > div.map > div.map_result > div > div.tab_conbox_wrap.traking_tab.is-scrolljs > div:nth-child(1) > div:nth-child(1) > div.program_info_contents > div.program_info_tit > h4')[0].text.strip()
        program["체험일"] = soup.select('#programTerm > option')[0].text
        program["체험주기"] = []
        for span in soup.select('#enrollDayWeek span.active'):
            program["체험주기"].append(span.text)
        program["체험이수시간"] = soup.select('#expCompltTime')[0].text.strip()
        if(len(soup.select('#expRunTime > span')) == 0):
            program["체험가능시간"] = ""
        else:
            program["체험가능시간"] = soup.select('#expRunTime > span')[0].text.strip()
        program["모집인원"] = soup.select('#recrtCnt')[0].text.strip()
        program["체험유형"] = soup.select('body > div.lay-body > div.map > div.map_result > div > div.tab_conbox_wrap.traking_tab.is-scrolljs > div:nth-child(1) > div:nth-child(1) > div.program_info_contents > div.dl_wrap > dl:nth-child(5) > dd')[0].text.strip()
        program["대상학교유형"] = soup.select('#schlType')[0].text.strip()
        program["체험대상"] = []
        for span in soup.select('body > div.lay-body > div.map > div.map_result > div > div.tab_conbox_wrap.traking_tab.is-scrolljs > div:nth-child(1) > div:nth-child(1) > div.program_info_contents > div.dl_wrap > dl:nth-child(7) > dd > div > span'):
            program["체험대상"].append(span.text)
        program["체험직무학과"] = soup.select('body > div.lay-body > div.map > div.map_result > div > div.tab_conbox_wrap.traking_tab.is-scrolljs > div:nth-child(1) > div:nth-child(1) > div.program_info_contents > div.dl_wrap > dl:nth-child(8) > dd')[0].text.strip()
        program["참가비"] = soup.select('#patcptnCostType')[0].text.strip()
        program["기관"] = soup.select('body > div.lay-body > div.map > div.map_result > div > div.tab_conbox_wrap.traking_tab.is-scrolljs > div:nth-child(1) > div:nth-child(1) > div.program_info_contents > div.program_info_conb > span')[0].text.strip()
        program["세부정보"] = soup.select('body > div.lay-body > div.map > div.map_result > div > div.tab_conbox_wrap.traking_tab.is-scrolljs > div:nth-child(1) > div.program_info.program_info_programinfo')
        programs.append(program)

        tab = driver.find_element(By.CSS_SELECTOR, 'body > div.lay-body > div.map > div.map_result > div > button')
        tab.click()
        time.sleep(0.5)
        print(p, i, program["title"])

2 1 베이킹
2 2 저작권강의
2 3 금융감독원 금융경제강의
2 4 나만의 폰 가방만들기
2 5 필통만들기
2 6 카드지갑만들기


In [ ]:
len(programs)

27

In [ ]:
for i in programs:
    i["세부정보"] = i["세부정보"][0].text
for i in programs:
    i["세부정보"] = clean_text(i["세부정보"])

In [ ]:
#pd.DataFrame(programs).to_csv('programs_2022_clean세부정보.csv', index = False)

## 2. 청소년활동정보서비스 크롤링

In [ ]:
# 웹드라이버를 실행합니다 (예: Chrome)
driver = webdriver.Chrome()

# 주어진 URL로 이동합니다.
url = "https://www.youth.go.kr/youth/act/actSearch/allActSearchLst.yt?sCrtfcText=&sSttemntText=&sort=rgsdehour&order=&page=1&rows=10&kCrtfcSn=&kFcltySn=634&kSnOne=366655&kProgrmseCode=001005&kSnTwo=0&kSttemntManageNo=366655&kSttemntSn=0&kCrtfcAt=N&kSttemntAt=N&kSttemntNo=&kExclnccrtfcAtchmnflId=&searchDtl=&search=&sActRelmcode=&sActRelmcode1=&sActRelmcode2=&sActRelmcode3=&sActRelmcode4=&sActRelmcode5=&sActRelmcode6=&sActRelmcode7=&sActRelmcode8=&sActRelmcode9=&sActRelmcode10=&sActRelmcode11=&sActRelmcode12=&nas.cmm.token.html.TOKEN=09b980862bf1966808dbb02eeedd9ab4&sKeyword=&sActCtprvnCode1=020014&sActSignguCode1=&sActCtprvnCode2=&sActSignguCode2=&sActCtprvnCode3=&sActSignguCode3=&sInsttTyCode=&sInsttTyCode2=&sTrget=&sAge=mskul&stayng=&pc=&acteraDe=acteraDeSelect&sActeraBeginDe=2022-01-01&sActeraEndDe=2022-12-31&sCurSearchFlag=Y&sChkSearchFlag=Y"
driver.get(url)

In [ ]:
programs = []

In [ ]:
for _ in range(5):
    for page in range(4,14):
        driver.find_element(By.CSS_SELECTOR, f'#listForm > div.pagenavi.mt10.txt-center > a:nth-child({page})').click()
        time.sleep(1)
        print(page, len(programs))
        for i in range(1,11):
            driver.find_element(By.CSS_SELECTOR, f'#listForm > ul > li:nth-child({i}) > div.act-name-box.st2 > dl > dt > a').click()
            time.sleep(0.5)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            program = {}
            try:
                program["제목"] = soup.select('#contentsBody > h2')[0].text.strip().split('\t')[-1]
            except:
                print("title error")
                continue
            for s in soup.select('#partcptnBbsForm > div > table > tbody > tr'):
                try:
                    key = s.select('th')[0].text.strip()
                    value = s.select('td')[0].text.strip()
                    program[key] = value

                    key = s.select('th')[1].text.strip()
                    value = s.select('td')[1].text.strip()
                    program[key] = value
                except:
                    pass
            try:
                program["활동개요"] = soup.select('#actInfo > div > div:nth-child(3)')[0].text
                program["활동목표 및 상세내용"] = soup.select('#actInfo > div > div:nth-child(6)')[0].text
                program["주소"] = soup.select('#skipMap1 > p')[0].text.strip()
            except:
                pass
            driver.find_element(By.CSS_SELECTOR, '#contentsBody > div.tab-st3-default.bar-blue > div > button:nth-child(2)').click()
            programs.append(program)
            time.sleep(0.5)

4 0
5 10


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(programs)

In [ ]:
df.head(1)

,제목,분류,인원 및 연령,지역,참가비,활동일시,모집기간,참가대상,차량운행,인증번호,...,숙박여부,반복여부,기관명,전화번호,담당자,관련사이트,팩스번호,활동개요,활동목표 및 상세내용,주소
0,"청소년동아리축제 ""청,비"" 콘서트",일반 수능후 동아리,500명 \n\t\t\t\t\t\t\t\t\t\t \n\n\n...,경상북도 상주시,무료,2022-12-10 ~ 10,상시모집,개별모집,미운행,-,...,-,-,상주시청소년수련관,0545358594,사혜경,http://youth.sangju.go.kr,054-537-6739,청소년동아리 축제,○ 일 시 : 12. 10.(토) 17:00~19:00 ○ 장 소 : ...,경상북도 상주시 계산동 490


### 위도, 경도 추가

In [ ]:
df["주소"] = df["주소"].fillna("")

In [ ]:
geolocoder = Nominatim(user_agent = 'South Korea')

In [ ]:
raws = df["주소"]
px = []; py = []
for raw in tqdm(raws):
    if raw == "":
        px.append(-1)
        py.append(-1)
        continue
    else:
        s = raw
        tx = -1; ty = -1
        while True:
            if s == '':
                break
            geo = geolocoder.geocode(s)
            s = ' '.join(s.split(' ')[:-1])
            try:
                tx = geo.latitude
                ty = geo.longitude
                break
            except:
                pass
            
        px.append(tx)
        py.append(ty)

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
df["위도"] = px
df["경도"] = py

In [ ]:
df = df[df["위도"] != -1]

### 텍스트 추가 처리

In [ ]:
df["활동목표 및 상세내용"] = df["활동목표 및 상세내용"].apply(clean_text)
df["활동개요"] = df["활동개요"].apply(clean_text)
df["인원 및 연령"] = df["인원 및 연령"].apply(lambda text: text.split('\t')[0].strip())

C:\Users\User\AppData\Local\Temp\ipykernel_22320\3953391486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["활동목표 및 상세내용"] = df["활동목표 및 상세내용"].apply(clean_text)
C:\Users\User\AppData\Local\Temp\ipykernel_22320\3953391486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["활동개요"] = df["활동개요"].apply(clean_text)
C:\Users\User\AppData\Local\Temp\ipykernel_22320\3953391486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [ ]:
df.head(1)

,제목,분류,인원 및 연령,지역,참가비,활동일시,모집기간,참가대상,차량운행,인증번호,...,기관명,전화번호,담당자,관련사이트,팩스번호,활동개요,활동목표 및 상세내용,주소,위도,경도
0,"청소년동아리축제 ""청,비"" 콘서트",일반 수능후 동아리,500명,경상북도 상주시,무료,2022-12-10 ~ 10,상시모집,개별모집,미운행,-,...,상주시청소년수련관,0545358594,사혜경,http://youth.sangju.go.kr,054-537-6739,청소년동아리 축제,일 시 토 장 소 상주시문화회관 대 상 중고등학교 청소...,경상북도 상주시 계산동 490,36.428149,128.170728


### 중요한 행만 뽑아서 저장

In [ ]:
df.rename(columns = {"인원 및 연령": "인원"}, inplace = True)
df_maincolumns = df[["제목", "인원", "참가비", "활동일시", "기관명", "활동개요", "활동목표 및 상세내용", "주소", "위도", "경도"]]

C:\Users\User\AppData\Local\Temp\ipykernel_22320\200665632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {"인원 및 연령": "인원"}, inplace = True)


In [ ]:
#df_maincolumns.to_csv('./data/청소년활동정보서비스_위도경도 추가.csv', index = False)